In [1]:
import json
import pandas as pd
import numpy as np
import re

### TASKS

Convert:
- Age to integer
- Market value strings to numeric
- Transfer fee strings to numeric
- Players nationalities to strings with 2 columns (nation 1, nation 2)

Add:
- Transfer type (loan, loan end, free, retire)

Store as JSON files per league

### DATA

In [2]:
def read_file(league, season, window):
    filename = '../data/{league}_{season}_{window}.json'.format(
        league=league,
        season=season,
        window=window
    )
    
    with open(filename, 'r') as f:
        data = json.load(f)

    res = []
    for team in data:
        in_ = pd.DataFrame(team['in'])
        if len(in_) > 0:
            in_.loc[:, 'dir'] = 'in'

        left = pd.DataFrame(team['left'])
        if len(left) > 0:
            left.loc[:, 'dir'] = 'left'

        team_df = pd.concat([in_, left])
        if len(team_df) > 0:
            team_df.loc[:, 'team_name'] = team['team']['team_name']
            team_df.loc[:, 'team_country'] = team['team']['team_country']
            team_df.loc[:, 'team_id'] = team['team']['team_id']
        
        res.append(team_df)
        
    res_df = pd.concat(res)
    res_df.loc[:, 'league'] = league
    res_df.loc[:, 'season'] = season
    res_df.loc[:, 'window'] = window
    
    return res_df

In [3]:
SCRAPE_LEAGUES = ['GB1', 'ES1', 'IT1', 'L1', 'FR1', 'PO1', 'NL1'] # GB1
SCRAPE_SEASONS = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
SCRAPE_WINDOWS = ["s", "w"]

transfers_data = pd.DataFrame([])

for league in SCRAPE_LEAGUES:
    for season in SCRAPE_SEASONS:
        for window in SCRAPE_WINDOWS:
            
            df = read_file(league, season, window)
            transfers_data = pd.concat([
                transfers_data,
                df
            ])

In [4]:
transfers_data.dtypes

counter_team_country     object
counter_team_id          object
counter_team_name        object
market_val               object
player_age               object
player_id                object
player_name              object
player_nat               object
player_pos               object
transfer_fee             object
transfer_id              object
dir                      object
team_name                object
team_country             object
team_id                  object
league                   object
season                  float64
window                   object
dtype: object

In [5]:
#transfers_data.head(3)

### PREPROCESS

In [6]:
# Age to integer
transfers_data.loc[:, 'player_age'] = pd.to_numeric(transfers_data.loc[:, 'player_age'], errors='coerce')

# Season to integer
transfers_data.loc[:, 'season'] = pd.to_numeric(transfers_data.loc[:, 'season'], errors='coerce', downcast='integer')

In [7]:
def parse_transfer_fee(transfer_fee_raw):
    amnt_pat = r"(€\d+\.*\d*)"
    amnt = re.findall(amnt_pat, transfer_fee_raw)
    
    if not amnt:
        return np.nan
    
    mult = 1000 if 'Th.' in transfer_fee_raw else 1000000
    
    return float(amnt[0].replace("€", "")) * mult

In [8]:
transfers_data.loc[:, 'transfer_fee_amnt'] = transfers_data.loc[:, 'transfer_fee'].apply(parse_transfer_fee)

transfers_data.loc[:, 'is_free'] = transfers_data.loc[:, 'transfer_fee'].str.contains("free")

transfers_data.loc[
    transfers_data.is_free == True,
    'transfer_fee_amnt'
] = 0

transfers_data.loc[:, 'is_loan'] = transfers_data.loc[:, 'transfer_fee'].str.lower().str.contains("loan")
transfers_data.loc[:, 'is_loan_end'] = transfers_data.loc[:, 'transfer_fee'].str.contains("End of loan")

transfers_data.loc[:, 'is_retired'] = transfers_data.counter_team_id.str.contains("Retired")

transfers_data.loc[:, 'market_val_amnt'] = transfers_data.loc[:, 'market_val'].apply(parse_transfer_fee)

In [16]:
def parse_nation(x):
    
    return pd.Series({
        'player_nation': x[0] if x else np.nan,
        'player_nation2': x[1] if len(x) > 1 else np.nan
    })

transfers_data = pd.concat([
    transfers_data,
    transfers_data.player_nat.apply(parse_nation)
], axis=1)

In [17]:
transfers_data.head()

,counter_team_country,counter_team_id,counter_team_name,market_val,player_age,player_id,player_name,player_nat,player_pos,transfer_fee,...,season,window,transfer_fee_amnt,is_free,is_loan,is_loan_end,is_retired,market_val_amnt,player_nation,player_nation2
0,England,1071,Wigan Athletic,-,23.0,33544,Antonio Valencia,[Ecuador],RB,€18.80m,...,2009,s,18800000.0,False,False,False,False,NaN,Ecuador,NaN
1,Norway,687,Molde FK,€1.60m,21.0,62049,Mame Diouf,[Senegal],CF,€4.50m,...,2009,s,4500000.0,False,False,False,False,1600000.0,Senegal,NaN
2,France,40,FC Girondins Bordeaux,€400Th.,20.0,43261,Gabriel Obertan,"[France, Guadeloupe]",AM,€4.00m,...,2009,s,4000000.0,False,False,False,False,400000.0,France,Guadeloupe
3,England,762,Newcastle United,-,29.0,1397,Michael Owen,[England],CF,free transfer,...,2009,s,0.0,True,False,False,False,NaN,England,NaN
4,England,5242,Manchester United U18,-,18.0,73538,Scott Moffatt,[England],CB,-,...,2009,s,NaN,False,False,False,False,NaN,England,NaN


### STORE DATASET

In [18]:
dataset_cols = [
        'league',
        'season',
        'window',
        'team_id',
        'team_name',
        'team_country',
        'dir',
        'player_id',
        'player_name',
        'player_age',
        'player_nation',
        'player_nation2',
        'player_pos',
        'counter_team_id',
        'counter_team_name',
        'counter_team_country',
        'transfer_fee_amnt',
        'market_val_amnt',
        'is_free',
        'is_loan',
        'is_loan_end',
        'is_retired',
        'transfer_id',
]

transfers_dataset = transfers_data.loc[
    :,
    dataset_cols
]

In [19]:
transfers_dataset.to_csv('../dataset/transfers.csv', index=False)